# Explaining My Script

The first thing I did was import the packages needed for my script:

In [1]:
import keras as kr
import tensorflow as tf
from keras.models import Sequential
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.models import load_model
import sklearn.preprocessing as pre
import sys
import gzip
import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageDraw, ImageTk
import tkinter as tk

C:\Users\dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Next I added a function which builds my neural network or model:

In [2]:
def buildModel():

    model = kr.models.Sequential()

    ## https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/
    ## Used this reference for dropout values as I was unsure why this was used 
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))                            
    model.add(Dropout(0.2))

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Opens our train images and labels to train our model
    with gzip.open('dataset/train-images-idx3-ubyte.gz', 'rb') as f:
        train_img = f.read()

    with gzip.open('dataset/train-labels-idx1-ubyte.gz', 'rb') as f:
        train_lbl = f.read()
    
    ## Convert both sets of images and labels to arrays of and divide the images by 255 to get the bits between 0-1
    train_img =  np.array(list(train_img[16:])).reshape(60000, 28, 28).astype(np.uint8)/ 255.0
    train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)
    
    ## Convert our labels to binary numbers , e.g 1 = 100000000 , 2 = 010000000
    encoder = pre.LabelBinarizer()
    encoder.fit(train_lbl)
    outputs = encoder.transform(train_lbl)
    
    ## Reshape our array to fit model input
    inputs = train_img.reshape(60000, 784)


    ## Train the model with our inputs(Images) and outputs (Labels)
    model.fit(inputs, outputs, epochs=5, batch_size=128)
    
    ## Save our model as Mnist for loading it in later instead of building our model every single time
    model.save('Mnist')
    
    print(str(ModelCreated))

Next I had to add a function which makes our image ready for input to our model:

In [3]:
def convertImage(imagefile):

    ## Really good reference for this : http://www.appstate.edu/~marshallst/GLY3455/lectures/9_Image_Processing.pdf

    ## Convert to greyscale
    im = Image.open(imagefile).convert('L')

    ## Make sure image is resized
    im= im.resize((28, 28), Image.BICUBIC)

    ## Convert to list
    im = list(im.getdata())

    # Currently everything is in bytes 0 - 255 , we want to make this 0-1 
    im = [(255 - x) * 1.0 / 255.0 for x in im]
    
    ## need to reshape for our model, expects an array of length 1-D array of size 784
    im =  np.array(list(im)).reshape(1,784)

    print("Image successfully converted! Sending To model")

    ## Send the ready array to our build model function
    CompareImage(im)

After this I added a function which takes in our ready image which is now an array of size 784, loads our model , gets the labels converted using labelBinarizer() so we can convert back to what the model thinks the number is:

In [4]:
def CompareImage(imageFile):
    ## Load in our labels once again for converting back to numbers
    with gzip.open('dataset/train-labels-idx1-ubyte.gz', 'rb') as f:
        train_lbl = f.read()
    train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)

    ## Load our model
    New_model = load_model('Mnist')
    print("According to my network your number is: ")
    
    ## Get numbers as binary format once again
    encoder = pre.LabelBinarizer()
    encoder.fit(train_lbl)
    
    ## Give prediction
    print(encoder.inverse_transform(New_model.predict(imageFile)))

As an extra feature I found a youtube video which used tkinter to create a drawable canvas which also saved the image you drew. I referenced this video in my code.

In [5]:
## Canvas all taken from https://www.youtube.com/watch?v=OdDCsxfI8S0
width = 200
height = 200
center = height//2
white = (255, 255, 255)
green = (0,128,0)
ModelCreated = False

def save():
    filename = "image.png"
    image1.save(filename)

def paint(event):
    # python_green = "#476042"
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    cv.create_oval(x1, y1, x2, y2, fill="black",width=5)
    draw.line([x1, y1, x2, y2],fill="black",width=5)

##############################################################################################

Then by using a simple while and if else statements I created a simple command line menu for my program. I displayed the menu items in another function.

In [6]:
def print_menu():       
    print("-" * 15 , "Welcome to Keiths Digit Recognition Script" , 15 * "-")
    print("A. Create Model (Must do this first) " + "Model Created: " + str(ModelCreated))
    print("B. Select your own image")
    print("C. Draw your digit")
    print("D. Exit")
   
loop=True      
  
while loop:          ## While loop which will keep going until loop = False
    print_menu()    ## Displays menu
    choice = input("Enter your choice [A-C]: ")
    print(choice)
     
    if choice == 'A':
        print("Creating Model")
        buildModel()
        ModelCreated = True
    elif choice == 'B':     
        userInput = input("Please enter file name/path: ")
        convertImage(userInput)
    elif choice=='C':
        # Canvas taken from https://www.youtube.com/watch?v=OdDCsxfI8S0
        print("Creating canvas (X canvas off when finished and select option one and enter 'image.png')")
        root = tk.Tk()

        # Tkinter create a canvas to draw on
        cv = tk.Canvas(root, width=width, height=height, bg='white')
        cv.pack()

        # PIL create an empty image and draw object to draw on
        # memory only, not visible
        image1 = PIL.Image.new("RGB", (width, height), white)
        draw = ImageDraw.Draw(image1)

        # do the Tkinter canvas drawings (visible)
        # cv.create_line([0, center, width, center], fill='green')

        cv.pack()
        cv.bind("<B1-Motion>", paint)

        # do the PIL image/draw (in memory) drawings
        # draw.line([0, center, width, center], green)

        # PIL image can be saved as .png .jpg .gif or .bmp file (among others)
        # filename = "my_drawing.png"
        # image1.save(filename)
        button=tk.Button(text="save",command=save)
        button.pack()
        root.mainloop()
    elif choice=='D':
        print("Exit")
        
        ## You can add your code or functions here
        loop=False # This will make the while loop to end as not value of loop is set to False
    else:
        # Any integer inputs other than values 1-5 we print an error message
        print("Wrong option selection. Enter any key to try again..")

--------------- Welcome to Keiths Digit Recognition Script ---------------
A. Create Model (Must do this first) Model Created: False
B. Select your own image
C. Draw your digit
D. Exit
Enter your choice [A-C]: A
A
Creating Model
Epoch 1/5
60000/60000 [==============================] - 20s 334us/step - loss: 0.2474 - acc: 0.9253
Epoch 2/5
60000/60000 [==============================] - 19s 321us/step - loss: 0.0996 - acc: 0.9695
Epoch 3/5
60000/60000 [==============================] - 19s 324us/step - loss: 0.0711 - acc: 0.9767
Epoch 4/5
60000/60000 [==============================] - 19s 323us/step - loss: 0.0559 - acc: 0.9819
Epoch 5/5
60000/60000 [==============================] - 19s 320us/step - loss: 0.0443 - acc: 0.9854
False
--------------- Welcome to Keiths Digit Recognition Script ---------------
A. Create Model (Must do this first) Model Created: True
B. Select your own image
C. Draw your digit
D. Exit
Enter your choice [A-C]: B
B
Please enter file name/path: Four.png
Image suc

As we can see, First I have to Select Option A: Create Model , when this is done I change the value of model created in the menu to true as you can see.

Next we select Option B: Select our own image, here we can either enter the name of the image as long as it is in the same directory as the script, otherwise we must give the full location path.

Option C: Draw your own image does work if you run the actual script from your command line , however it doesnt run on jupyter notebook. When you draw your number and press save it saves the file as "image.png" in the same directory of the script. You then just have to select Option B: Select your own image and enter "image.png" to test your newly drawn image.

